In [3]:
import pandas as pd
import sqlite3

In [4]:
path_search = 'MyData/SearchQueries.json'
df_search = pd.read_json(path_search)
df_search.head()

,platform,searchTime,searchQuery,searchInteractionURIs
0,ANDROID,2020-02-07T12:49:26.512Z[UTC],Moonlight,[spotify:track:6L4F4btXioJPhEwz6T7YYt]
1,ANDROID,2020-02-08T21:44:06.888Z[UTC],mc guime,[spotify:artist:3ge4xOaKvWfhRwgx0Rldov]
2,ANDROID,2020-02-09T20:52:42.231Z[UTC],My Only One (No Hay Nadie Más) Sebastián Yatra,[]
3,ANDROID,2020-02-13T01:15:16.958Z[UTC],idont,[spotify:track:41zXlQxzTi6cGAjpOXyLYH]
4,ANDROID,2020-02-15T13:52:02.476Z[UTC],ted talk,[spotify:show:1VXcH8QHkjRcTCEd88U3ti]


Como temos três arquivos de Streaming, vou concatenar todos em um DF só.

In [5]:
paths_streaming = ['MyData/StreamingHistory0.json', 'MyData/StreamingHistory1.json', 'MyData/StreamingHistory2.json']

df_streaming = pd.DataFrame([])

for path in paths_streaming:
    df_temp = pd.read_json(path)
    df_streaming = pd.concat([df_streaming, df_temp])

df_streaming.head()

,endTime,artistName,trackName,msPlayed
0,2019-04-28 14:19,MC L da Vinte,Parado no Bailão,25390
1,2019-04-30 14:25,MC Kevin o Chris,Dentro do Carro,50800
2,2019-05-01 09:54,Mc Livinho,Espelho No Teto,192947
3,2019-05-01 09:58,UCLÃ,Pa Pa Pa,230357
4,2019-05-01 10:03,MC L da Vinte,Parado no Bailão,5630


Salvando em um SQLite, para ficar mais fácil de ler as informações:

In [6]:
connection = sqlite3.connect('data/database_spotify.db')
cursor = connection.cursor()

In [16]:
# SEARCH
cursor.execute('CREATE TABLE SEARCH (platform, searchTime, searchQuery, searchInteractionURIs)')
connection.commit()

df_search.to_sql('SEARCH', connection, if_exists='replace', index=False)

## testing the results
cursor.execute('''  
SELECT * FROM SEARCH LIMIT 10
          ''')

df_temp = DataFrame(cursor.fetchall(), columns=['platform', 'searchTime', 'searchQuery', 'searchInteractionURIs'])    
print(df_temp)

InterfaceError: Error binding parameter 3 - probably unsupported type.

In [7]:
# STREAMING
# cursor.execute('CREATE TABLE STREAMING (endTime, artistName, trackName, msPlayed)')
# connection.commit()
# df_streaming.to_sql('STREAMING', connection, if_exists='replace', index=False)

## testing the results
cursor.execute('''  
SELECT * FROM STREAMING LIMIT 10
          ''')

df_temp = pd.DataFrame(cursor.fetchall(), columns=['platform', 'searchTime', 'searchQuery', 'searchInteractionURIs'])  

df_temp.head()

,platform,searchTime,searchQuery,searchInteractionURIs
0,2019-04-28 14:19,MC L da Vinte,Parado no Bailão,25390
1,2019-04-30 14:25,MC Kevin o Chris,Dentro do Carro,50800
2,2019-05-01 09:54,Mc Livinho,Espelho No Teto,192947
3,2019-05-01 09:58,UCLÃ,Pa Pa Pa,230357
4,2019-05-01 10:03,MC L da Vinte,Parado no Bailão,5630
